# Clone git repo and install requirements

In [ ]:
!git clone https://github.com/XPixelGroup/HAT.git

Cloning into 'HAT'...
remote: Enumerating objects: 410, done.
remote: Counting objects: 100% (251/251), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 410 (delta 196), reused 155 (delta 122), pack-reused 159
Receiving objects: 100% (410/410), 20.73 MiB | 23.95 MiB/s, done.
Resolving deltas: 100% (226/226), done.


In [ ]:
%cd HAT

/content/HAT


In [ ]:
!pip install -r requirements.txt
!python setup.py develop

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 22.9 MB/s eta 0:00:00
  Created wheel for basicsr: filename=basicsr-1.3.4.9-py3-none-any.whl size=194420 sha256=be0c47510f1e37a4f7572a4dc6678dcd6d3a62546f1bd264df2433ef1e61c1c2
  Stored in directory: /root/.cache/pip/wheels/37/19/97/8206d928857f2ef0625cb621b689005610515b4841be4e5ddb
Successfully built basicsr
/usr/local/lib/python3.10/dist-packages/setuptools/__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements sh

In [ ]:
!pwd

/content/HAT


# Process data

## Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Process train dataset

In [ ]:
!unzip /content/drive/MyDrive/Datasets/train/DIV2K_train_HR.zip -d /content/HAT/datasets/DIV2K
!unzip /content/drive/MyDrive/Datasets/train/DIV2K_train_LR_bicubic_X4.zip -d /content/HAT/datasets/DIV2K

In [ ]:
# naming train dataset
import os
for filename in os.listdir("/content/HAT/datasets/DIV2K/DIV2K_train_LR_bicubic/X4"):
    old_file_path = os.path.join("/content/HAT/datasets/DIV2K/DIV2K_train_LR_bicubic/X4", filename)
    new_filename = filename[0:4] + filename[6:]
    new_file_path = os.path.join("/content/HAT/datasets/DIV2K/DIV2K_train_LR_bicubic/X4", new_filename)
    os.rename(old_file_path, new_file_path)

In [ ]:
!git clone https://github.com/cszn/BSRGAN.git

Cloning into 'BSRGAN'...
remote: Enumerating objects: 541, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 541 (delta 144), reused 105 (delta 101), pack-reused 386
Receiving objects: 100% (541/541), 168.90 MiB | 32.83 MiB/s, done.
Resolving deltas: 100% (196/196), done.


In [ ]:
from torchvision.transforms import v2
import torch
import os
import random
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt
import numpy as np

def randomAffine(img):
    transf = v2.RandomAffine(degrees=0, shear=(-10, 10))
    if random.random() < 0.25:
        img = transf(img)
    return img

def data_augmentation(img):
    transforms = v2.Compose([
        v2.RandomHorizontalFlip(p=0.5), # Lật ảnh theo chiều ngang với xác suất 0.5
        v2.RandomVerticalFlip(p=0.5),  # Lật ảnh theo chiều dọc với xác suất 0.5
        v2.ColorJitter(brightness=0.2, contrast=0.2),  # Thay đổi độ sáng và tương phản
        v2.Lambda(lambda x: randomAffine(x)),       # Biến đổi shear
        v2.RandomErasing(p=0.5, scale=(0.01, 0.1), ratio=(0.3, 3.3)),  # Tạo nhiễu ngẫu nhiên
    ])
    return transforms(img)

def patch_extraction(image_file_list, image_HR_dir, is_display=False):
    """
        image_file_list: list of 4 images (only name of image, not path), [img_1.jpg, img_2.jpg, img_3.png, img_4.jpeg]
        image_HR_dir: original HR image directory

        Return: list of 16 patch images
    """
    patch_list = []
    patch_aug_list = []
    for file_name in image_file_list:
        img = Image.open(os.path.join(image_HR_dir, file_name))
        width, height = img.size

        patch_width = width // 2  # Chia đều chiều rộng
        patch_height = height // 2  # Chia đều chiều cao

        # Cắt hình ảnh thành bốn phần và thêm vào patch list
        patch_list.append(img.crop((0, 0, patch_width, patch_height)))
        patch_list.append(img.crop((patch_width, 0, width, patch_height)))
        patch_list.append(img.crop((0, patch_height, patch_width, height)))
        patch_list.append(img.crop((patch_width, patch_height, width, height)))
    for patch in patch_list:
        patch_aug_list.append(data_augmentation(patch))

    if is_display:
        fig, axes = plt.subplots(4, 4, figsize=(12, 12))
        for i in range(4):
            for j in range(4):
                k = i * 4 + j
                if k < len(patch_aug_list):
                    axes[i, j].imshow(patch_aug_list[k])
                    axes[i, j].axis('off')  # Tắt trục x, y
                    axes[i, j].set_title(f"Image {k}")
        plt.show()

    return patch_aug_list
def random_crop_image(img, crop_width, crop_height):
    width, height = img.size
    x = random.randint(0, width - crop_width)
    y = random.randint(0, height - crop_height)
    cropped_image = img.crop((x, y, x + crop_width, y + crop_height))
    return cropped_image

def mosaic_generation(patch_aug_list, image_HR_dir, image_LR_dir, file_name_saved, is_display=False):
    selected_patch_index = random.sample(range(len(patch_aug_list)), k=4) # Chọn ngẫu nhiên 4 hình trong tập ảnh
    patchs = []
    for index in selected_patch_index:
        patchs.append(patch_aug_list[index])
    width, height = [], []          # w0 -> w3, h0 -> h3
    for patch in patchs:
        w, h = patch.size
        width.append(w)
        height.append(h)
    w0 = w2 = min(width[0], width[2])
    h0 = h1 = min(height[0], height[1])
    w1 = w3 = min(width[1], width[3])
    h2 = h3 = min(height[2], height[3])
    croppeds = patchs
    croppeds[0] = random_crop_image(patchs[0], w0, h0)
    croppeds[1] = random_crop_image(patchs[1], w1, h1)
    croppeds[2] = random_crop_image(patchs[2], w2, h2)
    croppeds[3] = random_crop_image(patchs[3], w3, h3)

    if is_display:
        fig, axes = plt.subplots(2, 4, figsize=(12, 12))
        for j in range(4):
            axes[0, j].imshow(patchs[j])
            axes[0, j].axis('off')  # Tắt trục x, y
            axes[0, j].set_title(f"Image {j}")
        for j in range(4):
            axes[1, j].imshow(croppeds[j])
            axes[1, j].axis('off')  # Tắt trục x, y
            axes[1, j].set_title(f"Image {j + 4}")
        plt.show()

    new_width = w0 + w1
    new_height = h0 + h2
    result_image = Image.new("RGB", (new_width, new_height))
    result_image.paste(croppeds[0], (0, 0))
    result_image.paste(croppeds[1], (w0, 0))
    result_image.paste(croppeds[2], (0, h0))
    result_image.paste(croppeds[3], (w0, h0))
    new_width = new_width - (new_width % 12)
    new_height = new_height - (new_height % 12)
    HR_image = result_image.crop((0, 0, new_width, new_height))
    HR_image.save(os.path.join(image_HR_dir, file_name_saved))
    print(file_name_saved, "HR has been saved! Image size: ", HR_image.size)

    LR_image = result_image.resize((new_width//4, new_height//4), resample=Image.BICUBIC)
    LR_image.save(os.path.join(image_LR_dir, file_name_saved))
    print(file_name_saved, "LR has been saved! Image size: ", LR_image.size)
    return True

def patch_Mosaic(image_HR_dir, image_LR_dir, scale=4, num_of_gen=200):
    """
        image_HR_dir: Đường dẫn thư mục chứa bộ data High Resolution
        image_LR_dir: Đường dẫn thư mục chứa bộ data Low Resolution
        scale: Tỷ lệ sinh ảnh Low Resolution
        num_of_gen: Số lượng ảnh mới cần sinh
    """
    list_image_names = os.listdir(image_HR_dir)  # Lấy dir của toàn bộ tập ảnh
    total_image = len(list_image_names)   # Tổng số lượng ảnh trong tập ảnh
    for i in range(num_of_gen):
        image_file_list = random.choices(list_image_names, k=4)   # Chọn ngẫu nhiên 4 hình trong tập ảnh
        patch_aug_list = patch_extraction(image_file_list, image_HR_dir)
        file_name_saved = f"{total_image + i + 1:04}.png"         # Đặt tên ảnh mới sinh (độ dài 4 theo như định dạng của bộ DIV2K)
        mosaic_generation(patch_aug_list, image_HR_dir, image_LR_dir, file_name_saved)

In [ ]:
patch_Mosaic(image_HR_dir="./datasets/DIV2K/DIV2K_train_HR", image_LR_dir="./datasets/DIV2K/DIV2K_train_LR_bicubic/X4", num_of_gen=10)

## Zip the new dataset and copy to google drive

In [ ]:
%cd /content/HAT/datasets/DIV2K

/content/HAT/datasets/DIV2K


In [ ]:
!zip -r DIV2K_train_HR_Mosaic.zip DIV2K_train_HR/

  adding: DIV2K_train_HR/ (stored 0%)
  adding: DIV2K_train_HR/0480.png (deflated 0%)
  adding: DIV2K_train_HR/0540.png (deflated 0%)
  adding: DIV2K_train_HR/0257.png (deflated 0%)
  adding: DIV2K_train_HR/0646.png (deflated 0%)
  adding: DIV2K_train_HR/0842.png (deflated 0%)
  adding: DIV2K_train_HR/0511.png (deflated 0%)
  adding: DIV2K_train_HR/0644.png (deflated 0%)
  adding: DIV2K_train_HR/0570.png (deflated 0%)
  adding: DIV2K_train_HR/0569.png (deflated 0%)
  adding: DIV2K_train_HR/0457.png (deflated 0%)
  adding: DIV2K_train_HR/0608.png (deflated 0%)
  adding: DIV2K_train_HR/0663.png (deflated 0%)
  adding: DIV2K_train_HR/0566.png (deflated 0%)
  adding: DIV2K_train_HR/0313.png (deflated 0%)
  adding: DIV2K_train_HR/0483.png (deflated 0%)
  adding: DIV2K_train_HR/0797.png (deflated 0%)
  adding: DIV2K_train_HR/0993.png (deflated 0%)
  adding: DIV2K_train_HR/0545.png (deflated 0%)
  adding: DIV2K_train_HR/0090.png (deflated 0%)
  adding: DIV2K_train_HR/0694.png (deflated 0%)
  

In [ ]:
!zip -r DIV2K_train_LR_bicubic_X4_Mosaic.zip DIV2K_train_LR_bicubic/

  adding: DIV2K_train_LR_bicubic/ (stored 0%)
  adding: DIV2K_train_LR_bicubic/X4/ (stored 0%)
  adding: DIV2K_train_LR_bicubic/X4/0480.png (deflated 0%)
  adding: DIV2K_train_LR_bicubic/X4/0540.png (deflated 0%)
  adding: DIV2K_train_LR_bicubic/X4/0257.png (deflated 0%)
  adding: DIV2K_train_LR_bicubic/X4/0646.png (deflated 0%)
  adding: DIV2K_train_LR_bicubic/X4/0842.png (deflated 0%)
  adding: DIV2K_train_LR_bicubic/X4/0511.png (deflated 0%)
  adding: DIV2K_train_LR_bicubic/X4/0644.png (deflated 0%)
  adding: DIV2K_train_LR_bicubic/X4/0570.png (deflated 0%)
  adding: DIV2K_train_LR_bicubic/X4/0569.png (deflated 0%)
  adding: DIV2K_train_LR_bicubic/X4/0457.png (deflated 0%)
  adding: DIV2K_train_LR_bicubic/X4/0608.png (deflated 0%)
  adding: DIV2K_train_LR_bicubic/X4/0663.png (deflated 0%)
  adding: DIV2K_train_LR_bicubic/X4/0566.png (deflated 0%)
  adding: DIV2K_train_LR_bicubic/X4/0313.png (deflated 0%)
  adding: DIV2K_train_LR_bicubic/X4/0483.png (deflated 0%)
  adding: DIV2K_trai

In [ ]:
%cp -rf DIV2K_train_HR_Mosaic.zip /content/drive/MyDrive/Datasets/train/DIV2K_train_HR_Mosaic.zip
%cp -rf DIV2K_train_LR_bicubic_X4_Mosaic.zip /content/drive/MyDrive/Datasets/train/DIV2K_train_LR_bicubic_X4_Mosaic.zip